In [3]:
import dash
import dash_leaflet as dl
from shapely.geometry import Point, shape
from dash import html, Output, Input, dcc, State, dash_table
import osmnx as ox
import networkx as nx
import base64
import io
import time
import json
import geopandas as gpd
import pandas as pd 
from isochrone_generator import IsochroneGenerator

####

class DashApp:
    def __init__(self):
        self.app = dash.Dash(__name__)
        self.city_boundaries = {}
        self.city_graphs = {}
        self.setup_layout()
        self.register_callbacks()
        self.generator = None
        self.isochrone_names = {}

    ####

    def setup_layout(self):
        self.app.layout = html.Div([
            html.Div([
                # Left side (Map)
                html.Div([
                    # Loading spinner for the map (this will show while waiting for callback)
                    dcc.Loading(
                        id="loading-isochrone", 
                        type="circle",  # You can choose from 'circle', 'dot', or 'default'
                        overlay_style={"visibility": "visible", "filter": "blur(2px)"},
                        children=dl.Map(
                            id='map',
                            center=[52.778151, -1.840227],  # Initial map center
                            zoom=5,  # Initial zoom
                            style={'width': '100%', 'height': '80vh'},  
                            children=[
                                dl.LayersControl(
                                    children=[dl.BaseLayer(dl.TileLayer(), name='Base Layer', checked=True),
                                    ]
                                )
                            ]
                        )
                    )
                ], style={'flex': 2, 'height': '100vh'}),  # Left side takes more space
    
                # Right side (Form with buttons)
                html.Div([
                    html.Div([
                        dcc.Input(id='citynameid', placeholder='London, UK', type='text', value="", style={'width': '100%'})  # City name input
                    ], style={'padding': '10px 0'}),  
    
                    html.Div([
                        html.Button("Search", id="btn", style={'width': '100%'})  # Search button
                    ], style={'padding': '10px 0'}),
    
                    html.Div([
                        html.Button("Add boundary", id="boundarybtn", style={'width': '100%'})  # Add boundary button
                    ], style={'padding': '10px 0'}),  
                    
                    html.Div([
                        html.Div(children="Click on the Map to get Lat and Lon", id="textid", style={'width': '100%'})  
                    ], style={'padding': '10px 0'}),  

                    html.Div([
                        dcc.Input(id="isochrone_name_id", placeholder='Enter Isochrone name', value=None,  style={'width': '100%'})  
                    ], style={'padding': '10px 0'}),
                    
                    html.Div([
                        dcc.Input(id='latid', placeholder='Enter Latitude', type='number', value=None, style={'width': '27%', 'marginRight': '2%'}),
                        dcc.Input(id='lonid', placeholder='Enter Longitude', type='number', value=None, style={'width': '27%', 'marginRight': '2%'}),
                        dcc.Input(id='timeid', placeholder='Enter Time Limit', type='number', value=None, style={'width': '30%'})
                    ], style={'padding': '10px 0', 'display': 'flex', 'flexWrap': 'wrap'}),  # Flex to make inputs in one row

                    html.Div([
                        html.Button("Add isochrone", id="isochronebtn", style={'width': '100%'})  # Add isochrone button
                    ], style={'padding': '10px 0'}, title = 'An isochrone shows areas reachable within a set time from a location.'),
                    
                    html.Div([
                        dcc.Upload(id='upload_isochrone_data', children=html.Button('Upload Isochrone data File (CSV/Excel)', style={'width': '100%'}), multiple=False),
                        html.Div(id='output_data_upload', style={'padding': '10px'}),
                    ], style={'width': '100%'}),
                    
                    html.Div([
                    # Dropdown to select from the generated isochrones
                        dcc.Dropdown(id='isochrone_dropdown', options=[], placeholder="Select Isochrone", style={'width': '100%'}, searchable=False)
                    ], style={'padding': '10px 0'}),
    
                    html.Div([
                        html.Button("Add Shortest Paths", id="shortest_path_btn", style={'width': '100%'})  # Add shortest paths button
                    ], style={'padding': '10px 0'}, title = 'Shortest paths inside the isochrone show the quickest routes to reach destinations within the time limit.'),
    
                    html.Div([
                        html.Button("Add Road Network", id="road_network_btn", style={'width': '100%'})  # Add road network button
                    ], style={'padding': '10px 0'}, title = 'The road network inside the isochrone represents all accessible roads within the defined time area.'),
    
                ], style={'padding': '10px', 'height': '100vh', 'flex': 1})  # Set width and padding for the right panel
            ], style={'display': 'flex'}),  # Use flexbox to align left and right sections
    
            # ConfirmDialog to show the message
            dcc.ConfirmDialog(
                id='confirm_dialog',
                message='',
                displayed=False  
            ),

            dcc.Store(id='layers-store', data=[]),
            
        ])

    ####

    def register_callbacks(self):
        # Callback for searching and centering the map
        @self.app.callback(
            [Output("map", "viewport"),
             Output('confirm_dialog', 'displayed', allow_duplicate=True),  # Show confirm dialog
             Output('confirm_dialog', 'message', allow_duplicate=True),  # Message in confirm dialog
            ],
            Input("btn", "n_clicks"),  
            State("citynameid", "value"),  
            prevent_initial_call=True
        )
        def update_map_search(n_search_clicks, cityname):
            new_center = [52.778151, -1.840227]  
            zoom = 5
            dialog_message = ""  
            show_dialog = False   
    
            # If "Search" button is clicked, center the map on the city without adding boundaries
            if n_search_clicks:

                if cityname is None or cityname == '':
                    return dash.no_update, True, 'Please enter city name'
                elif cityname in self.city_boundaries:
                    city_boundary = self.city_boundaries[cityname]
                    lat, lon = shape(city_boundary['features'][0]['geometry']).centroid.y, shape(city_boundary['features'][0]['geometry']).centroid.x
                    return dict(center = [lat, lon], zoom=10, transition="flyTo"), False, ''

                if not cityname in self.city_boundaries:
                    try:
                        gdf = ox.geocode_to_gdf(cityname)
                        lat, lon = gdf.geometry.centroid.y.values[0], gdf.geometry.centroid.x.values[0]
                        return dict(center=[lat, lon], zoom=10, transition="flyTo"), False, ''
                    
                    except Exception as e:
                        return dash.no_update(), True, f"Error: {e}"
                
            return dash.no_update, show_dialog, dialog_message
            
        ####

        # Callback for adding the boundary to the map
        @self.app.callback(
            [Output('layers-store', 'data', allow_duplicate=True),
             Output('confirm_dialog', 'displayed', allow_duplicate=True),
             Output('confirm_dialog', 'message', allow_duplicate=True), 
             Output('loading-isochrone', 'children', allow_duplicate=True)],
             
            Input("boundarybtn", "n_clicks"), 
            State("citynameid", "value"), 
            State('layers-store', 'data'),
            prevent_initial_call=True
        )
        def update_map_boundary(n_boundary_clicks, city_name, existing_layers ):
            dialog_message = ''
            show_dialog = False
            
            if n_boundary_clicks:
                if city_name is None or city_name == '':
                    return dash.no_update, True, 'Please enter a Valid city name', dash.no_update

                elif any(layer.get('props', {}).get('id') == f'{city_name}_boundary' for layer in existing_layers):
                    return dash.no_update, True, 'Boundary already added!', dash.no_update
                
                elif not any(layer.get('props', {}).get('id') == f'{city_name}_boundary' for layer in existing_layers):
                    try:
                        loading_state = dcc.Loading(type='circle', children=html.Div("Loading boundary..."))
                        # Get the boundary as a GeoDataFrame
                        gdf = ox.geocode_to_gdf(city_name)
                        geojson_data = gdf.geometry.to_json()  
                        geojson_dict = json.loads(geojson_data)  
                        new_layer = dl.Overlay(dl.GeoJSON( data=geojson_dict, style={"color": "black", "weight": 2, "opacity": 1, "fillColor": "white", "fillOpacity": 0.2}
                                                    ), name=f'{city_name}_boundary', id=f'{city_name}_boundary', checked=True)
                        
                        existing_layers.append(new_layer)
                        self.city_boundaries[city_name] = geojson_dict
                        
                        # If city graph does not exist, generate it
                        if city_name not in self.city_graphs:
                            self.generate_graph(city_name)
                            
                    except Exception as e:
                        dialog_message = f"Error: {e}"  
                        show_dialog = True  
                        
                    return existing_layers, show_dialog, dialog_message, dash.no_update
                        
            return dash.no_update, show_dialog, dialog_message, dash.no_update


        # Define the callback to update the map when the button is clicked
        @self.app.callback(
            [Output('layers-store', 'data', allow_duplicate=True),
             Output('confirm_dialog', 'displayed', allow_duplicate=True),
             Output('confirm_dialog', 'message', allow_duplicate=True), 
             Output('loading-isochrone', 'children', allow_duplicate=True), 
             Output('isochrone_dropdown', 'options', allow_duplicate=True)
            ],
            State('citynameid', 'value'),
            State('isochrone_name_id', 'value'),
            State('latid', 'value'),
            State('lonid', 'value'),
            State('timeid', 'value'),
            Input('isochronebtn', 'n_clicks'),
            State('layers-store', 'data'),
            State('isochrone_dropdown', 'options'),
            prevent_initial_call=True,
        )
        def update_isochrone(cityname, isochrone_name, lat, lon, time, n_clicks, existing_layers, current_options):
            # Ensure that when the button is clicked, all fields are checked
            if n_clicks > 0:
                if not any(layer.get('props', {}).get('id') == f'{cityname}_boundary' for layer in existing_layers):
                    return dash.no_update, True, 'Add boundary first!', dash.no_update, dash.no_update
                
                # If any of the fields are empty, show an error message
                elif not lat or not lon or not time or not cityname or not isochrone_name:
                    return dash.no_update, True, 'Null or Invalid input', dash.no_update, dash.no_update
                
                elif any(layer.get('props', {}).get('name') == isochrone_name for layer in existing_layers):
                    return dash.no_update, True, 'Isochrone already added!', dash.no_update, dash.no_update

                elif any(layer.get('props', {}).get('name') == isochrone_name for layer in existing_layers):
                    return dash.no_update, True, 'Please Change Isochrone name!', dash.no_update, dash.no_update
        
                    
                cityboundary = self.city_boundaries.get(cityname)
                polygon = shape(cityboundary['features'][0]['geometry'])

                # Check if the point is inside the polygon
                if not polygon.contains(Point(lon, lat)):
                    return dash.no_update, True, 'The point is outside the city boundary!', dash.no_update, dash.no_update 
  
                # If all fields are valid, proceed with isochrone generation
                if not any(layer.get('props', {}).get('id') == isochrone_name for layer in existing_layers) and cityname in self.city_graphs:

                    loading_state = dcc.Loading(type='circle', children=html.Div("Loading Isochrone..."))
                    
                    # Generate new isochrone
                    isochrone = self.generator.generate_isochrone(isochrone_name, lat, lon, time)
                    
                    # Convert the isochrone to a GeoDataFrame and then to GeoJSON
                    new_isochrone_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[isochrone])
                    new_geojson_data = new_isochrone_gdf.geometry.to_json()

                    #number = len([layer for layer in existing_layers if 'isochrone' in layer.get('props', {}).get('id')]) + 1 
        
                    # Return the new GeoJSON data to the map
                    new_layer = dl.Overlay(dl.GeoJSON( data=json.loads(new_geojson_data), style={"color": "red", "weight": 2, "opacity": 1, "fillColor": "white", "fillOpacity": 0.2}
                                                    ), name=isochrone_name, id = isochrone_name, checked=True)
                        
                    existing_layers.append(new_layer)

                    new_layer = dl.Overlay(dl.Marker(position=[lat, lon]), name = f'Marker : {isochrone_name}', id=f'{isochrone_name}_marker', checked = True)
                    existing_layers.append(new_layer)

                    updated_options = current_options + [{'label': isochrone_name, 'value': isochrone_name}]
                 
                    return existing_layers, False, "", dash.no_update, updated_options 
        
            # Return empty GeoJSON if the button hasn't been clicked yet
            return existing_layers, False, "", dash.no_update, updated_options

        @self.app.callback(
            Output('output_data_upload', 'children'),
            Output('isochrone_dropdown', 'options', allow_duplicate=True),
            Output('layers-store', 'data', allow_duplicate=True),
            Output('loading-isochrone', 'children', allow_duplicate=True),
            Input('upload_isochrone_data', 'contents'),
            State('upload_isochrone_data', 'filename'),
            State('isochrone_dropdown', 'options'),
            State('layers-store', 'data'),
            prevent_initial_call=True
        )
        def upload_file(contents, filename, current_options, existing_layers):
            if contents is None:
                return dash.no_update, dash.no_update, dash.no_update, dash.no_update
        
            # Decode the uploaded file
            content_type, content_string = contents.split(',')
            decoded = base64.b64decode(content_string)
            try:
                if filename.endswith('.csv'):
                    # If it's a CSV file
                    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
                elif filename.endswith('.xls') or filename.endswith('.xlsx'):
                    # If it's an Excel file
                    df = pd.read_excel(io.BytesIO(decoded))
                else:
                    return "Unsupported file format. Please upload a CSV or Excel file.", dash.no_update, dash.no_update, dash.no_update
        
                # Ensure required columns exist in the uploaded file
                required_columns = ['Isochrone Name', 'Latitude', 'Longitude', 'Time']
                if not all(col in df.columns for col in required_columns):
                    return "The file must contain columns: Isochrone Name, Latitude, Longitude, Time", dash.no_update, dash.no_update, dash.no_update

                loading_state = dcc.Loading(type='circle', children=html.Div("Loading Isochrone..."))
        
                # Loop through each row and generate isochrones
                updated_options = current_options.copy()
                for _, row in df.iterrows():
                    isochrone_name = row['Isochrone Name']
                    lat = row['Latitude']
                    lon = row['Longitude']
                    time = row['Time']
        
                    # Validate the input data
                    if pd.isna(isochrone_name) or pd.isna(lat) or pd.isna(lon) or pd.isna(time):
                        return f"Missing or invalid data in row: {row}", dash.no_update, dash.no_update, dash.no_update
        
                    # Generate isochrone for this row
                    if not any(layer.get('props', {}).get('name') == isochrone_name for layer in current_options):
                        isochrone = self.generator.generate_isochrone(isochrone_name, lat, lon, time)
        
                        # Convert the isochrone to a GeoDataFrame and then to GeoJSON
                        new_isochrone_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[isochrone])
                        new_geojson_data = new_isochrone_gdf.geometry.to_json()
        
                        new_layer = dl.Overlay(dl.GeoJSON(data=json.loads(new_geojson_data), style={"color": "red", "weight": 2, "opacity": 1, "fillColor": "white", "fillOpacity": 0.2}),
                                               name=isochrone_name, id=isochrone_name, checked=True)
                        
                        existing_layers.append(new_layer)

                        new_layer = dl.Overlay(dl.Marker(position=[lat, lon]), name = f'Marker : {isochrone_name}', id=f'{isochrone_name}_marker', checked = True)
                        existing_layers.append(new_layer)
                        
                        updated_options.append({'label': isochrone_name, 'value': isochrone_name})
        
                return "File uploaded and isochrones generated successfully!", updated_options, existing_layers, dash.no_update
        
            except Exception as e:
                return f"Error processing file: {str(e)}", dash.no_update, dash.no_update, dash.no_update
        
        ####

        # Define the callback to update the latitude and longitude when the map is clicked
        @self.app.callback(
            [Output('confirm_dialog', 'displayed', allow_duplicate=True),
             Output('confirm_dialog', 'message', allow_duplicate=True),
             Output('latid', 'value'),
             Output('lonid', 'value')
            ],
            [State('citynameid', 'value'),  
             Input('map', 'clickData')
            ], 
             prevent_initial_call=True, # Listen for clicks on the map
        )
        def update_lat_lon(cityname, clickData):
  
            if clickData:
                # Extract the latitude and longitude from the clickData
                lat = clickData['latlng']['lat']
                lon = clickData['latlng']['lng']
                
                # Create a Point object from the latitude and longitude
                point = Point(lon, lat)

                if not cityname in self.city_boundaries:
                    return True, 'Please add boundary first!', None, None
                     
                # Get the boundary of the selected city
                cityboundary = self.city_boundaries.get(cityname)
    
                # Convert the GeoJSON to a Shapely Polygon
                polygon = shape(cityboundary['features'][0]['geometry'])
                        
                # Check if the point is inside the polygon
                if polygon.contains(point):
                    return False, '', lat, lon
                else:
                    return True, 'The point is outside the city boundary!', None, None
                        
            # Return None, None if no click or city boundary is not found
            return False, '', None, None

         ####
        @self.app.callback(
            [Output('layers-store', 'data', allow_duplicate=True),
            Output('confirm_dialog', 'displayed', allow_duplicate=True),  
            Output('confirm_dialog', 'message', allow_duplicate=True),
            Output('loading-isochrone', 'children', allow_duplicate=True)], 
            Input('shortest_path_btn', 'n_clicks'),
            State('layers-store', 'data'),
            State('isochrone_dropdown', 'value'),
            prevent_initial_call=True
        )
        def add_shortest_path(n_clicks, existing_layers, isochrone_name):
            if n_clicks is not None and n_clicks > 0:
                
                if not any(layer.get('props', {}).get('name') == isochrone_name for layer in existing_layers):
                    return dash.no_update, True, 'Isochrone is not added. Please add an isochrone first.', dash.no_update
                    
                elif any(layer.get('props', {}).get('id') == f'{isochrone_name}_shortest_path' for layer in existing_layers):
                    return dash.no_update, True, 'Shortest Paths for this Isochrone is already added', dash.no_update
                    
                else:
                    loading_state = dcc.Loading(type='circle', children=html.Div("Loading Shortest Path..."))
                    shortest_paths = self.generator.generate_shortest_paths(isochrone_name)
                    shortest_paths_geojson = { 'type': 'FeatureCollection', 'features': shortest_paths }
                    new_layer = dl.Overlay(dl.GeoJSON(data = shortest_paths_geojson), name=f'SP : {isochrone_name}', id=f'{isochrone_name}_shortest_path', checked = True)
                    existing_layers.append(new_layer)
                    
            return existing_layers, False, '', dash.no_update

        ####
        @self.app.callback(
            [Output('layers-store', 'data', allow_duplicate=True),
            Output('confirm_dialog', 'displayed', allow_duplicate=True),  
            Output('confirm_dialog', 'message', allow_duplicate=True),
            Output('loading-isochrone', 'children', allow_duplicate=True)],
            Input('road_network_btn', 'n_clicks'),
            State('layers-store', 'data'),
            State('isochrone_dropdown', 'value'),
            prevent_initial_call=True
        )
        def add_road_network(n_clicks, existing_layers, isochrone_name):
            if n_clicks is not None and n_clicks > 0:
                
                if not any(layer.get('props', {}).get('id') == isochrone_name for layer in existing_layers):
                    return dash.no_update, True, 'Isochrone is not added. Please add an isochrone first.', dash.no_update
                    
                elif any(layer.get('props', {}).get('id') == f'{isochrone_name}_network' for layer in existing_layers):
                    return dash.no_update, True, 'Road Network for this Isochrone is already added.', dash.no_update
                    
                else:
                    loading_state = dcc.Loading(type='circle', children=html.Div("Loading Road Network..."))
                    road_network = self.generator.generate_road_network(isochrone_name)
                    new_layer = dl.Overlay(dl.GeoJSON(data=road_network), name = f'RN : {isochrone_name}', id = f'{isochrone_name}_network', checked=True)
                    existing_layers.append(new_layer)
                    
            return existing_layers, False, '', dash.no_update
            
        # Callback to update the map layers based on the store
        @self.app.callback(
            Output('map', 'children'),
            Input('layers-store', 'data'),
            prevent_initial_call=True
        )
        def update_map_layers(layers):
            base_layer = dl.BaseLayer(dl.TileLayer(), name='base', checked=True)
            
            # Only include checked layers
            #checked_layers = [layer for layer in layers if layer.get('props', {}).get('checked', True)]
            
            layers_control = dl.LayersControl(
                children=[base_layer] + layers  
            )
            
            return [layers_control]

                  
    # Generate graph
    def generate_graph(self, city_name: str):
        if city_name not in self.city_graphs:
            try:
                if city_name == 'Somerset, UK':
                    self.generator = IsochroneGenerator(graph_path = 'Somerset_UK_drive.graphml')
                    self.city_graphs[city_name] = self.generator.G
                else:
                    self.generator = IsochroneGenerator(place_name = city_name)
                    self.city_graphs[city_name] = self.generator.G
                    return self.city_graphs[city_name]  
                
            except Exception as e:
                # If an error occurs, handle it and return the error message
                error_message = f"Failed to create graph for {city_name}. Error: {str(e)}"
                return error_message  # Return the error message instead of raising an exception
                
        # If the graph already exists in the dictionary, return it directly
        return self.city_graphs.get(city_name)        
            
# Run the app
if __name__ == '__main__':
    app_instance = DashApp()
    app_instance.app.run_server(debug=True)